In [31]:
from dotenv import load_dotenv
load_dotenv()

True

In [32]:
#Imports
from google import genai
from tavily import TavilyClient
import os
import re
import json
import sqlite3
from datetime import datetime
from typing import List, Dict, Any, Optional
import numpy as np
from pathlib import Path
from typing import Callable, Any, Dict, Optional
from memory_system import (
    MemoryStore, MemoryRetriever, FeedbackHandler, 
    ContextBuilder, AgentMemoryIntegration
)
from reflection_system import ReflectionWorkflow

In [62]:
# Model Configuration
MODEL_PROVIDER = "google"  # Options: "google", "fathom", "openai", etc.
MODEL_NAME = "gemini-2.5-pro"  # Default model name

# Model-specific configurations
MODEL_CONFIGS = {
    "google": {
        "api_key_env": "GOOGLE_API_KEY",
        "default_model": "gemini-2.5-pro"
    },
    "fathom": {
        "api_key_env": "FATHOM_API_KEY",
        "default_model": "fathom-1",  # Adjust to your model name
        "base_url": "https://api.fathom.ai/v1"  # Adjust to your API endpoint
    },
    "openai": {
        "api_key_env": "OPENAI_API_KEY",
        "default_model": "gpt-4"
    }
}

In [63]:
class UnifiedLLMClient:
    """Wrapper to support multiple LLM providers with a unified interface."""
    
    def __init__(self, provider: str, model: str = None):
        self.provider = provider.lower()
        self.config = MODEL_CONFIGS.get(self.provider)
        
        if not self.config:
            raise ValueError(f"Unsupported provider: {provider}")
        
        self.model = model or self.config["default_model"]
        self.api_key = os.environ.get(self.config["api_key_env"])
        
        if not self.api_key:
            raise ValueError(f"API key not found for {provider}. Set {self.config['api_key_env']} in .env")
        
        self._initialize_client()
    
    def _initialize_client(self):
        """Initialize the appropriate client based on provider."""
        if self.provider == "google":
            from google import genai
            self.client = genai.Client(api_key=self.api_key)
            
        elif self.provider == "fathom":
            # If Fathom uses OpenAI-compatible API
            from openai import OpenAI
            self.client = OpenAI(
                api_key=self.api_key,
                base_url=self.config.get("base_url")
            )
            
        elif self.provider == "openai":
            from openai import OpenAI
            self.client = OpenAI(api_key=self.api_key)
    
    def generate_content(self, contents: str, config: dict = None):
        """Unified method to generate content across providers."""
        if self.provider == "google":
            return self.client.models.generate_content(
                model=self.model,
                contents=contents,
                config=config
            )
        
        elif self.provider in ["fathom", "openai"]:
            # OpenAI-style API
            system_instruction = config.get('system_instruction', '') if config else ''
            
            messages = []
            if system_instruction:
                messages.append({"role": "system", "content": system_instruction})
            messages.append({"role": "user", "content": contents})
            
            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages
            )
            
            # Wrap response to match Google's format
            class ResponseWrapper:
                def __init__(self, text):
                    self.text = text
            
            return ResponseWrapper(response.choices[0].message.content)

In [64]:
# Create tavily client
#client = genai.Client(api_key=os.environ.get('GOOGLE_API_KEY'))
#model="gemini-2.5-pro"
tavily_client = TavilyClient(api_key=os.environ.get('TAVILY_API_KEY')) # Initialize Tavily client

# Initialize unified client
unified_client = UnifiedLLMClient(
    provider=MODEL_PROVIDER,
    model=MODEL_NAME
)

# Initialize memory system
memory_store = MemoryStore("pm_agent_memory.db")
retriever = MemoryRetriever(memory_store)
feedback_handler = FeedbackHandler(memory_store)
context_builder = ContextBuilder(retriever)
memory_integration = AgentMemoryIntegration(
    memory_store, retriever, feedback_handler, context_builder
)

In [34]:
#Agents

class LeadArchitectAgent:
    """Lead Architect/Engineer Agent that designs system architecture and development plans."""
    
    def __init__(self, model: str, client, memory_integration: AgentMemoryIntegration, num_reflections: int = 0):
        self.model = model
        self.client = client
        self.memory_integration = memory_integration
        self.conversation_history = []
        self.num_reflections = num_reflections
        self.reflection_workflow = ReflectionWorkflow(client, model, num_reflections)
        self.system_instruction = """You are a Lead Software Architect and Engineering Manager with deep expertise in:
        - System architecture and design patterns (microservices, event-driven, monolithic, etc.)
        - Backend and frontend technologies (Python, JavaScript/TypeScript, React, Node.js, etc.)
        - Database design (SQL, NoSQL, data modeling, scaling strategies)
        - Cloud infrastructure (AWS, GCP, Azure) and DevOps practices
        - API design (REST, GraphQL, gRPC)
        - Security best practices and compliance
        - Performance optimization and scalability
        - Code quality, testing strategies, and CI/CD
        - Effort estimation and sprint planning
        - Which engineering skillsets are needed for different tasks (frontend, backend, data engineering, DevOps, etc.)
        - Technical debt management
        - Team coordination and technical leadership

        You analyze product requirements, UX designs, and statistical constraints to:
        1. Design robust, scalable system architectures
        2. Recommend appropriate technology stacks
        3. Create detailed technical specifications
        4. Provide realistic effort estimates (in story points or hours)
        5. Identify technical risks and dependencies
        6. Suggest phased implementation approaches
        7. Consider maintainability, security, and performance from the start
        
        You communicate technical concepts clearly to both technical and non-technical stakeholders,
        and always balance ideal solutions with practical constraints like timeline, budget, and team skills.
        
        If you are provided with search results, memories, or reflections, incorporate relevant information into your response, but don't acknowledge them explicitly."""
    
    def __call__(self, requirements: str, num_reflections: Optional[int] = None, verbose: bool = False):
        """
        Design architecture and development plan based on requirements.
        
        Args:
            requirements: The requirements/specs (from user, PM, UX, or statistician)
            num_reflections: Number of reflections for this specific query
            verbose: If True, returns reflection data
            
        Returns:
            (response, memory_id) tuple or (response, memory_id, reflection_data) if verbose
        """
        reflections_to_use = num_reflections if num_reflections is not None else self.num_reflections
        
        # Get memory context
        memory_context = self.memory_integration.pre_run_hook(requirements, "architecture_design")

        # Check if web search needed for technologies or patterns
        decision_prompt = f"""Analyze this architecture/engineering request and determine if you need to search for 
recent technologies, frameworks, design patterns, or best practices.

Request: {requirements}

Respond with ONLY "YES" or "NO":"""

        decision_response = self.client.models.generate_content(
            model=self.model,
            contents=decision_prompt
        )
        
        needs_search = "YES" in decision_response.text.strip().upper()
        
        search_context = ""
        if needs_search:
            print("🔍 Searching for architecture patterns and technologies...")
            search_results = search_web(f"software architecture best practices {requirements[:100]}")
            if isinstance(search_results, list):
                search_context = "\n\nTechnology & Architecture Research:\n"
                for i, result in enumerate(search_results, 1):
                    search_context += f"\n{i}. {result.get('title', '')}\n{result.get('content', '')}\n"
        
        # Build conversation context
        conversation_context = ""
        if self.conversation_history:
            conversation_context = "\n\nPrevious architecture decisions:\n"
            for entry in self.conversation_history[-3:]:
                conversation_context += f"Requirements: {entry['requirements'][:200]}...\nArchitecture: {entry['response'][:300]}...\n\n"

        def generate_response(query: str) -> str:
            full_query = (
                memory_context +
                conversation_context + 
                f"\nDesign system architecture and development plan for:\n{query}\n\n"
                "Structure your response with:\n"
                "1. **System Architecture Overview**: High-level architecture diagram description and key components\n"
                "2. **Technology Stack**: Recommended technologies with justifications\n"
                "3. **Data Architecture**: Database design, data flows, and storage strategies\n"
                "4. **API Design**: Endpoints, data contracts, and integration points\n"
                "5. **Security & Compliance**: Authentication, authorization, data protection\n"
                "6. **Development Plan**: Phased implementation approach with milestones\n"
                "7. **Effort Estimates**: Story points or time estimates per component/phase\n"
                "8. **Technical Risks**: Potential challenges and mitigation strategies" +
                search_context
            )
            
            response = self.client.models.generate_content(
                model=self.model,
                contents=full_query,
                config={'system_instruction': self.system_instruction}
            )
            return response.text
        
        if reflections_to_use > 0:
            print(f"🤔 Reflecting {reflections_to_use} time(s)...")
            self.reflection_workflow.num_reflections = reflections_to_use
            result = self.reflection_workflow.execute(
                initial_query=requirements,
                generate_response_fn=generate_response,
                system_instruction=self.system_instruction
            )
            final_response = result['final_response']
            reflection_data = result
        else:
            final_response = generate_response(requirements)
            reflection_data = None

        self.conversation_history.append({
            'requirements': requirements,
            'response': final_response
        })
        
        memory_id = self.memory_integration.post_run_hook(
            requirements,
            final_response,
            memory_type="architecture_design",
            metadata={
                'search_used': needs_search,
                'num_reflections': reflections_to_use
            }
        )

        if verbose and reflection_data:
            return final_response, memory_id, reflection_data
        
        return final_response, memory_id
    
    def set_reflections(self, num_reflections: int):
        self.num_reflections = num_reflections
        self.reflection_workflow.num_reflections = num_reflections
        print(f"Default reflection count updated to: {num_reflections}")
    
    def apply_feedback(self, memory_id: int, feedback_type: str, feedback_content: str = ""):
        self.memory_integration.apply_feedback(memory_id, feedback_type, feedback_content)
        print(f"Feedback '{feedback_type}' applied to memory {memory_id}")
        
    def clear_history(self):
        self.conversation_history = []
        self.reflection_workflow.clear_history()
        print("Architecture history cleared.")

class StatisticianAgent:
    """Statistician Agent that provides statistical validation, methodology suggestions, and risk analysis."""
    
    def __init__(self, model: str, client, memory_integration: AgentMemoryIntegration, num_reflections: int = 0):
        self.model = model
        self.client = client
        self.memory_integration = memory_integration
        self.conversation_history = []
        self.num_reflections = num_reflections
        self.reflection_workflow = ReflectionWorkflow(client, model, num_reflections)
        self.system_instruction = """You are an expert Statistician and Data Scientist with deep expertise in:
        - Experimental design and A/B testing methodologies
        - Causal inference techniques (propensity score matching, instrumental variables, difference-in-differences, RCTs)
        - Statistical hypothesis testing and power analysis
        - Observational study design and confounding control
        - Sample size calculations and statistical power
        - Bayesian and frequentist inference
        - Time series analysis and longitudinal data methods
        - Machine learning validation and model evaluation
        - Data quality assessment and bias detection
        - Privacy-preserving statistical methods
        - Communicating statistical concepts to non-technical stakeholders

        You analyze product plans, research designs, and data strategies to:
        1. Validate statistical assumptions and methodological soundness
        2. Suggest appropriate statistical techniques and experimental designs
        3. Flag potential risks, biases, and confounding factors
        4. Provide specific, actionable recommendations with clear rationale
        5. Assess feasibility of causal claims and required data infrastructure
        
        You communicate complex statistical concepts clearly, always considering practical constraints
        like sample size, data collection feasibility, and computational resources.
        If you are provided with search results, memories, or reflections, incorporate relevant information into your response, but don't acknowledge them explicitly."""
    
    def __call__(self, analysis_request: str, num_reflections: Optional[int] = None, verbose: bool = False):
        """
        Analyze plans/designs from a statistical perspective.
        
        Args:
            analysis_request: The plan, design, or question to analyze (from user, PM, or UX researcher)
            num_reflections: Number of reflections for this specific query
            verbose: If True, returns reflection data
            
        Returns:
            (response, memory_id) tuple or (response, memory_id, reflection_data) if verbose
        """
        reflections_to_use = num_reflections if num_reflections is not None else self.num_reflections
        
        # Get memory context
        memory_context = self.memory_integration.pre_run_hook(analysis_request, "statistical_analysis")

        # Check if web search needed for recent statistical methods or research
        decision_prompt = f"""Analyze this statistical analysis request and determine if you need to search for 
recent statistical methodologies, causal inference techniques, or research papers.

Request: {analysis_request}

Respond with ONLY "YES" or "NO":"""

        decision_response = self.client.models.generate_content(
            model=self.model,
            contents=decision_prompt
        )
        
        needs_search = "YES" in decision_response.text.strip().upper()
        
        search_context = ""
        if needs_search:
            print("🔍 Searching for statistical methods and research...")
            search_results = search_web(f"statistical methodology causal inference {analysis_request[:100]}")
            if isinstance(search_results, list):
                search_context = "\n\nRecent Statistical Methods & Research:\n"
                for i, result in enumerate(search_results, 1):
                    search_context += f"\n{i}. {result.get('title', '')}\n{result.get('content', '')}\n"
        
        # Build conversation context
        conversation_context = ""
        if self.conversation_history:
            conversation_context = "\n\nPrevious statistical analyses:\n"
            for entry in self.conversation_history[-3:]:
                conversation_context += f"Request: {entry['request']}\nAnalysis: {entry['response'][:300]}...\n\n"

        def generate_response(query: str) -> str:
            full_query = (
                memory_context +
                conversation_context + 
                f"\nProvide statistical analysis for:\n{query}\n\n"
                "Structure your response with:\n"
                "1. **Validation of Assumptions**: What statistical assumptions are being made?\n"
                "2. **Recommended Techniques**: Which specific statistical methods should be used?\n"
                "3. **Risk Warnings**: What are the key statistical risks and limitations?\n"
                "4. **Implementation Details**: Sample sizes, power calculations, data requirements\n"
                "5. **Causal Inference Considerations**: If applicable, how to establish causality" +
                search_context
            )
            
            response = self.client.models.generate_content(
                model=self.model,
                contents=full_query,
                config={'system_instruction': self.system_instruction}
            )
            return response.text
        
        if reflections_to_use > 0:
            print(f"🤔 Reflecting {reflections_to_use} time(s)...")
            self.reflection_workflow.num_reflections = reflections_to_use
            result = self.reflection_workflow.execute(
                initial_query=analysis_request,
                generate_response_fn=generate_response,
                system_instruction=self.system_instruction
            )
            final_response = result['final_response']
            reflection_data = result
        else:
            final_response = generate_response(analysis_request)
            reflection_data = None

        self.conversation_history.append({
            'request': analysis_request,
            'response': final_response
        })
        
        memory_id = self.memory_integration.post_run_hook(
            analysis_request,
            final_response,
            memory_type="statistical_analysis",
            metadata={
                'search_used': needs_search,
                'num_reflections': reflections_to_use
            }
        )

        if verbose and reflection_data:
            return final_response, memory_id, reflection_data
        
        return final_response, memory_id
    
    def set_reflections(self, num_reflections: int):
        self.num_reflections = num_reflections
        self.reflection_workflow.num_reflections = num_reflections
        print(f"Default reflection count updated to: {num_reflections}")
    
    def apply_feedback(self, memory_id: int, feedback_type: str, feedback_content: str = ""):
        self.memory_integration.apply_feedback(memory_id, feedback_type, feedback_content)
        print(f"Feedback '{feedback_type}' applied to memory {memory_id}")
        
    def clear_history(self):
        self.conversation_history = []
        self.reflection_workflow.clear_history()
        print("Statistical analysis history cleared.")

class UXResearcherAgent:
    """UX Researcher Agent that conducts user research based on product briefs."""
    
    def __init__(self, model: str, client, memory_integration: AgentMemoryIntegration, num_reflections: int = 0):
        self.model = model
        self.client = client
        self.memory_integration = memory_integration
        self.conversation_history = []
        self.num_reflections = num_reflections
        self.reflection_workflow = ReflectionWorkflow(client, model, num_reflections)
        self.system_instruction = """You are an experienced UX Researcher with expertise in:
        - User interviews and surveys
        - Usability testing and evaluation
        - User personas and journey mapping
        - Information architecture
        - Behavioral analysis and user psychology
        - A/B testing and experimentation
        - Accessibility and inclusive design
        - Analytics interpretation
        - Presenting research findings and recommendations

        You analyze product briefs and provide detailed UX research plans, methodologies, 
        and insights. You ask clarifying questions about target users, use cases, and 
        constraints. You base your recommendations on established UX principles and research methods.
        If you are provided with search results, memories, or reflections, incorporate relevant information into your response, but don't acknowledge them explicitly."""
    
    def __call__(self, brief: str, num_reflections: Optional[int] = None, verbose: bool = False):
        """
        Process a product brief and provide UX research recommendations.
        
        Args:
            brief: The product brief (from user or PM agent)
            num_reflections: Number of reflections for this specific query
            verbose: If True, returns reflection data
            
        Returns:
            (response, memory_id) tuple or (response, memory_id, reflection_data) if verbose
        """
        reflections_to_use = num_reflections if num_reflections is not None else self.num_reflections
        
        # Get memory context
        memory_context = self.memory_integration.pre_run_hook(brief, "ux_research")

        # Check if web search needed for UX best practices
        decision_prompt = f"""Analyze this UX research brief and determine if you need to search for current UX best practices, research methodologies, or industry standards.

Brief: {brief}

Respond with ONLY "YES" or "NO":"""

        decision_response = self.client.models.generate_content(
            model=self.model,
            contents=decision_prompt
        )
        
        needs_search = "YES" in decision_response.text.strip().upper()
        
        search_context = ""
        if needs_search:
            print("🔍 Searching for UX research best practices...")
            search_results = search_web(f"UX research methodology {brief[:100]}")
            if isinstance(search_results, list):
                search_context = "\n\nUX Research Best Practices:\n"
                for i, result in enumerate(search_results, 1):
                    search_context += f"\n{i}. {result.get('title', '')}\n{result.get('content', '')}\n"
        
        # Build conversation context
        conversation_context = ""
        if self.conversation_history:
            conversation_context = "\n\nPrevious research context:\n"
            for entry in self.conversation_history[-3:]:
                conversation_context += f"Brief: {entry['brief']}\nResearch Plan: {entry['response']}\n\n"

        def generate_response(query: str) -> str:
            full_query = (
                memory_context +
                conversation_context + 
                f"\n{query}" +
                search_context
            )
            
            response = self.client.models.generate_content(
                model=self.model,
                contents=full_query,
                config={'system_instruction': self.system_instruction}
            )
            return response.text
        
        if reflections_to_use > 0:
            print(f"🤔 Reflecting {reflections_to_use} time(s)...")
            self.reflection_workflow.num_reflections = reflections_to_use
            result = self.reflection_workflow.execute(
                initial_query=brief,
                generate_response_fn=generate_response,
                system_instruction=self.system_instruction
            )
            final_response = result['final_response']
            reflection_data = result
        else:
            final_response = generate_response(brief)
            reflection_data = None

        self.conversation_history.append({
            'brief': brief,
            'response': final_response
        })
        
        memory_id = self.memory_integration.post_run_hook(
            brief,
            final_response,
            memory_type="ux_research",
            metadata={
                'search_used': needs_search,
                'num_reflections': reflections_to_use
            }
        )

        if verbose and reflection_data:
            return final_response, memory_id, reflection_data
        
        return final_response, memory_id
    
    def set_reflections(self, num_reflections: int):
        self.num_reflections = num_reflections
        self.reflection_workflow.num_reflections = num_reflections
        print(f"Default reflection count updated to: {num_reflections}")
    
    def apply_feedback(self, memory_id: int, feedback_type: str, feedback_content: str = ""):
        self.memory_integration.apply_feedback(memory_id, feedback_type, feedback_content)
        print(f"Feedback '{feedback_type}' applied to memory {memory_id}")
        
    def clear_history(self):
        self.conversation_history = []
        self.reflection_workflow.clear_history()
        print("Research history cleared.")

class SampleDataGeneratorAgent:
    """Agent that generates sample/mock data for testing and prototyping."""
    
    def __init__(self, model: str, client, memory_integration: AgentMemoryIntegration, num_reflections: int = 0):
        self.model = model
        self.client = client
        self.memory_integration = memory_integration
        self.conversation_history = []
        self.num_reflections = num_reflections
        self.reflection_workflow = ReflectionWorkflow(client, model, num_reflections)
        self.system_instruction = """You are a Sample Data Generator specialist with expertise in:
        - Creating realistic test data and mock datasets
        - Understanding data schemas and structures
        - Generating diverse, representative data samples
        - Ensuring data privacy and anonymization
        - Creating edge cases and boundary conditions
        - Generating data in various formats (JSON, CSV, SQL, etc.)
        - Synthetic data generation techniques
        - Data volume scaling considerations

        You generate realistic, contextually appropriate sample data based on specifications.
        You ensure data quality, diversity, and adherence to constraints. You can generate
        data for user profiles, transactions, interactions, or any other entities needed
        for testing and prototyping.
        If you are provided with search results, memories, or reflections, incorporate relevant information into your response, but don't acknowledge them explicitly."""
    
    def __call__(self, specification: str, num_reflections: Optional[int] = None, verbose: bool = False):
        """
        Generate sample data based on specification.
        
        Args:
            specification: Description of data to generate (schema, format, volume, etc.)
            num_reflections: Number of reflections for this specific query
            verbose: If True, returns reflection data
            
        Returns:
            (response, memory_id) tuple or (response, memory_id, reflection_data) if verbose
        """
        reflections_to_use = num_reflections if num_reflections is not None else self.num_reflections
        
        memory_context = self.memory_integration.pre_run_hook(specification, "data_generation")

        # Build conversation context
        conversation_context = ""
        if self.conversation_history:
            conversation_context = "\n\nPrevious data generation context:\n"
            for entry in self.conversation_history[-3:]:
                conversation_context += f"Spec: {entry['spec']}\n\n"

        def generate_response(query: str) -> str:
            full_query = (
                memory_context +
                conversation_context + 
                f"\nGenerate sample data based on: {query}\n\n"
                "Provide the data in a structured format with clear labels and explanations."
            )
            
            response = self.client.models.generate_content(
                model=self.model,
                contents=full_query,
                config={'system_instruction': self.system_instruction}
            )
            return response.text
        
        if reflections_to_use > 0:
            print(f"🤔 Reflecting {reflections_to_use} time(s)...")
            self.reflection_workflow.num_reflections = reflections_to_use
            result = self.reflection_workflow.execute(
                initial_query=specification,
                generate_response_fn=generate_response,
                system_instruction=self.system_instruction
            )
            final_response = result['final_response']
            reflection_data = result
        else:
            final_response = generate_response(specification)
            reflection_data = None

        self.conversation_history.append({
            'spec': specification,
            'response': final_response
        })
        
        memory_id = self.memory_integration.post_run_hook(
            specification,
            final_response,
            memory_type="data_generation",
            metadata={'num_reflections': reflections_to_use}
        )

        if verbose and reflection_data:
            return final_response, memory_id, reflection_data
        
        return final_response, memory_id
    
    def set_reflections(self, num_reflections: int):
        self.num_reflections = num_reflections
        self.reflection_workflow.num_reflections = num_reflections
        print(f"Default reflection count updated to: {num_reflections}")
    
    def apply_feedback(self, memory_id: int, feedback_type: str, feedback_content: str = ""):
        self.memory_integration.apply_feedback(memory_id, feedback_type, feedback_content)
        print(f"Feedback '{feedback_type}' applied to memory {memory_id}")
        
    def clear_history(self):
        self.conversation_history = []
        self.reflection_workflow.clear_history()
        print("Data generation history cleared.")

class ProductManagerAgent:
    """Product Manager Agent with conversation memory and long-term learning."""
    
    def __init__(self, model: str, client, memory_integration: AgentMemoryIntegration, num_reflections: int = 0):
        self.model = model
        self.client = client
        self.memory_integration = memory_integration
        self.conversation_history = []
        self.num_reflections = num_reflections
        self.reflection_workflow = ReflectionWorkflow(client, model, num_reflections)
        self.system_instruction = """You are an experienced Product Manager with expertise in:
        - Product strategy and roadmap planning
        - Market research and competitive analysis
        - User research and requirements gathering
        - Feature prioritization and backlog management
        - Stakeholder communication
        - Metrics and KPI definition
        - Go-to-market strategies
        - Agile methodologies
        - Providing feedback to your product specialist, engineering, design, data science, testing, and marketing teams

        You provide practical, actionable advice and ask clarifying questions when needed.
        When provided with search results or past memories, incorporate relevant information into your response and cite sources.
        If you are provided with search results, memories, or reflections, incorporate relevant information into your response, but don't acknowledge them explicitly."""
    
    def __call__(self, user_query: str, num_reflections: Optional[int] = None, verbose: bool = False):
        """
        Make the agent callable like a function.
        
        Args:
            user_query: The user's query
            num_reflections: Number of reflections for this specific query (overrides default)
            verbose: If True, returns reflection data
            
        Returns:
            (response, memory_id) tuple or (response, memory_id, reflection_data) if verbose
        """
        # Use query-specific reflections or fall back to instance default
        reflections_to_use = num_reflections if num_reflections is not None else self.num_reflections
        
        # Get memory context
        memory_context = self.memory_integration.pre_run_hook(user_query, "pm_interaction")

        # Decide if search is needed
        decision_prompt = f"""Analyze this query and determine if you need web search.

Query: {user_query}

Respond with ONLY "YES" or "NO":"""

        decision_response = self.client.models.generate_content(
            model=self.model,
            contents=decision_prompt
        )
        
        needs_search = "YES" in decision_response.text.strip().upper()
        
        # Perform search if needed
        search_context = ""
        if needs_search:
            print("🔍 Searching the web...")
            search_results = search_web(user_query)
            if isinstance(search_results, list):
                search_context = "\n\nWeb Search Results:\n"
                for i, result in enumerate(search_results, 1):
                    search_context += f"\n{i}. {result.get('title', '')}\n{result.get('content', '')}\n"
        
        # Build conversation context
        conversation_context = ""
        if self.conversation_history:
            conversation_context = "\n\nPrevious conversation:\n"
            for entry in self.conversation_history[-5:]:  # Keep last 5 exchanges
                conversation_context += f"User: {entry['user']}\nAssistant: {entry['assistant']}\n\n"

        # Define response generation function for reflection workflow
        def generate_response(query: str) -> str:
            full_query = (
                memory_context +
                conversation_context + 
                f"\n{query}" +
                search_context
            )
            
            response = self.client.models.generate_content(
                model=self.model,
                contents=full_query,
                config={'system_instruction': self.system_instruction}
            )
            return response.text
        
        # Execute with or without reflection
        if reflections_to_use > 0:
            print(f"🤔 Reflecting {reflections_to_use} time(s)...")
            # Temporarily update reflection workflow
            self.reflection_workflow.num_reflections = reflections_to_use
            result = self.reflection_workflow.execute(
                initial_query=user_query,
                generate_response_fn=generate_response,
                system_instruction=self.system_instruction
            )
            final_response = result['final_response']
            reflection_data = result
        else:
            final_response = generate_response(user_query)
            reflection_data = None

        # Store in conversation history
        self.conversation_history.append({
            'user': user_query,
            'assistant': final_response
        })
        
        # Log to long-term memory
        memory_id = self.memory_integration.post_run_hook(
            user_query,
            final_response,
            memory_type="pm_interaction",
            metadata={
                'search_used': needs_search, 
                'num_reflections': reflections_to_use, 
                'reflection_used': reflections_to_use > 0
            }
        )

        if verbose and reflection_data:
            return final_response, memory_id, reflection_data
        
        return final_response, memory_id
    
    def set_reflections(self, num_reflections: int):
        """Update the default number of reflections."""
        self.num_reflections = num_reflections
        self.reflection_workflow.num_reflections = num_reflections
        print(f"Default reflection count updated to: {num_reflections}")
    
    def apply_feedback(self, memory_id: int, feedback_type: str, 
                      feedback_content: str = ""):
        """Apply feedback to a specific interaction."""
        self.memory_integration.apply_feedback(memory_id, feedback_type, feedback_content)
        print(f"Feedback '{feedback_type}' applied to memory {memory_id}")
        
    def clear_history(self):
        """Clear conversation history."""
        self.conversation_history = []
        self.reflection_workflow.clear_history()
        print("Conversation history cleared.")

In [35]:
#Helper functions
def remove_markdown(response_text):
    """Remove markdown formatting and print clean text."""
    # Remove markdown formatting
    text = re.sub(r'\*\*(.+?)\*\*', r'\1', response_text)  # Bold
    text = re.sub(r'\*(.+?)\*', r'\1', text)  # Italic
    text = re.sub(r'`(.+?)`', r'\1', text)  # Code
    text = re.sub(r'#+\s', '', text)  # Headers
    return text
def chat_with_agent(agent, agent_name="Agent"):
    """
    Generic interactive chat interface for any agent.
    
    Args:
        agent: The agent instance (callable) to chat with
        agent_name: Display name for the agent (default: "Agent")
    
    Type 'exit', 'quit', or 'bye' to end the conversation.
    Type 'clear' to clear conversation history.
    """
    print("=" * 60)
    print(f"{agent_name} Chat")
    print("=" * 60)
    print(f"Start chatting with the {agent_name}!")
    print("Type 'exit', 'quit', or 'bye' to end the chat.")
    print("Type 'clear' to clear conversation history.\n")
    
    while True:
        user_input = input("You: ").strip()
        
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print(f"\n{agent_name}: Thanks for chatting! Goodbye!")
            break
        
        if user_input.lower() == 'clear':
            if hasattr(agent, 'clear_history'):
                agent.clear_history()
            continue
            
        if not user_input:
            continue
            
        try:
            response = agent(user_input)
            print(f"\n{agent_name}: {remove_markdown(response)}\n")
        except Exception as e:
            print(f"\nError: {e}\n")
            print("Please try again or type 'exit' to quit.\n")
def search_web(query):
    """Search the web using Tavily."""
    try:
        response = tavily_client.search(query, max_results=5)
        return response['results']
    except Exception as e:
        return f"Search error: {e}"



In [ ]:
# Create agent instances
pm_agent = ProductManagerAgent(model=MODEL_NAME, client=unified_client, memory_integration=memory_integration)
ux_researcher = UXResearcherAgent(model=MODEL_NAME, client=unified_client, memory_integration=memory_integration)
data_generator = SampleDataGeneratorAgent(model=MODEL_NAME, client=unified_client, memory_integration=memory_integration)
statistician = StatisticianAgent(model=MODEL_NAME, client=unified_client, memory_integration=memory_integration)
lead_architect = LeadArchitectAgent(model=MODEL_NAME, client=unified_client, memory_integration=memory_integration)

In [ ]:
# Query
query = "I want to create a project management tool for sales teams."

num_reflections = 0

In [53]:
#Workflow
response, memory_id = pm_agent(query, num_reflections=num_reflections)
ux_response, ux_memory_id = ux_researcher(response, num_reflections=num_reflections)
stats_analysis, stats_memory_id = statistician(
    f"Please analyze this product and UX research plan from a statistical perspective:\n\n"
    f"Product Plan: {response}\n\n"
    f"UX Research Plan: {ux_response}",
    num_reflections=num_reflections
)
arch_design, arch_memory_id = lead_architect(
    f"Please design the system architecture for this product:\n\n"
    f"Product Plan: {response}\n\n"
    f"UX Research: {ux_response}\n\n"
    f"Statistical Requirements: {stats_analysis}",
    num_reflections=num_reflections
)
user_stories, memory_id = pm_agent(f"Can you create a list of user stories with acceptance criteria given these discussions?:\n\n"
                               f"Product Plan: {response}\n\n"
                               f"UX Research: {ux_response}\n\n"
                            f"Statistical Requirements: {stats_analysis}"
                            f"Architecture and effort estimate: {arch_design}", 
                            num_reflections=num_reflections)
ux_ok, ux_memory_id = ux_researcher(f"Are you okay with these user stories and acceptance criteria?:\n\n{user_stories}",num_reflections=num_reflections)
stats_ok, stats_memory_id = statistician(f"Are the statistical requirements feasible given these user stories?:\n\n{user_stories}",num_reflections=num_reflections)
arch_ok, arch_memory_id = lead_architect(f"Is the architecture feasible given these user stories and acceptance criteria?:\n\n{user_stories}",num_reflections=num_reflections)
#print(remove_markdown(response))

🔍 Searching the web...
🤔 Reflecting 2 time(s)...
🔍 Searching for UX research best practices...
🤔 Reflecting 2 time(s)...
🔍 Searching for statistical methods and research...
🤔 Reflecting 2 time(s)...
🤔 Reflecting 2 time(s)...
🤔 Reflecting 2 time(s)...
🤔 Reflecting 2 time(s)...
🔍 Searching for statistical methods and research...
🤔 Reflecting 2 time(s)...
🤔 Reflecting 2 time(s)...


In [ ]:
strategy_prompt, memory_id = pm_agent(f"Please provide me with a prompt that summarizes the overall product plan, UX research, statistical requirements, architecture design, and roadmap that I can input to a tool called Gamma which generate powerpoints using GenAI:\n\n"
                               f"Product Plan: {response}\n\n"
                               f"UX Research: {ux_response}\n\n"
                            f"Statistical Requirements: {stats_analysis}"
                            f"Architecture and effort estimate: {arch_design}"
                            f"User Stories and Acceptance Criteria: {user_stories}"
                            f"UX Researcher Approval: {ux_ok}"
                            f"Statistician Approval: {stats_ok}"
                            f"Architect Approval: {arch_ok}", 
                            num_reflections=num_reflections)

engg_driven_prompt, memory_id = lead_architect(f"""Your task is to generate a single, comprehensive prompt that can be fed into Google AIStudio. 
This prompt should synthesize the following inputs into a coherent specification for prototype generation using GenAI/vibecoding:

- Overall Product Plan
- UX Research
- Statistical Requirements
- Architecture Design and Effort Estimate
- User Stories and Acceptance Criteria
- Approvals (UX Researcher, Statistician, Architect)

When constructing the AIStudio prompt:
1. Integrate all inputs into a structured, readable format that AIStudio can consume directly.
2. Explicitly require that the prototype be responsive across both web and mobile platforms.
3. Frame the output requirements so AIStudio generates a functional, interpretable, and extensible prototype.
4. Ensure the synthesized prompt emphasizes alignment with product goals, UX findings, statistical rigor, architectural scalability, and roadmap milestones.

Deliver the final output as a single, well‑formed prompt ready to be passed into AIStudio.
\n\n"""
                               f"Product Plan: {response}\n\n"
                               f"UX Research: {ux_response}\n\n"
                            f"Statistical Requirements: {stats_analysis}"
                            f"Architecture and effort estimate: {arch_design}"
                            f"User Stories and Acceptance Criteria: {user_stories}"
                            f"UX Researcher Approval: {ux_ok}"
                            f"Statistician Approval: {stats_ok}"
                            f"Architect Approval: {arch_ok}", 
                            num_reflections=num_reflections)

pm_driven_prompt, memory_id = pm_agent(f"""Your task is to generate a single, comprehensive prompt that can be fed into Google AIStudio. 
This prompt should synthesize the following inputs into a coherent specification for prototype generation using GenAI/vibecoding:

- Overall Product Plan
- UX Research
- Statistical Requirements
- Architecture Design and Effort Estimate
- User Stories and Acceptance Criteria
- Approvals (UX Researcher, Statistician, Architect)

When constructing the AIStudio prompt:
1. Integrate all inputs into a structured, readable format that AIStudio can consume directly.
2. Explicitly require that the prototype be responsive across both web and mobile platforms.
3. Frame the output requirements so AIStudio generates a functional, interpretable, and extensible prototype.
4. Ensure the synthesized prompt emphasizes alignment with product goals, UX findings, statistical rigor, architectural scalability, and roadmap milestones.

Deliver the final output as a single, well‑formed prompt ready to be passed into AIStudio.
\n\n"""
                               f"Product Plan: {response}\n\n"
                               f"UX Research: {ux_response}\n\n"
                            f"Statistical Requirements: {stats_analysis}"
                            f"Architecture and effort estimate: {arch_design}"
                            f"User Stories and Acceptance Criteria: {user_stories}"
                            f"UX Researcher Approval: {ux_ok}"
                            f"Statistician Approval: {stats_ok}"
                            f"Architect Approval: {arch_ok}", 
                            num_reflections=num_reflections)

ux_driven_prompt, ux_memory_id = ux_researcher(f"""Your task is to generate a single, comprehensive prompt that can be fed into Google AIStudio. 
This prompt should synthesize the following inputs into a coherent specification for prototype generation using GenAI/vibecoding:

- Overall Product Plan
- UX Research
- Statistical Requirements
- Architecture Design and Effort Estimate
- User Stories and Acceptance Criteria
- Approvals (UX Researcher, Statistician, Architect)

When constructing the AIStudio prompt:
1. Integrate all inputs into a structured, readable format that AIStudio can consume directly.
2. Explicitly require that the prototype be responsive across both web and mobile platforms.
3. Frame the output requirements so AIStudio generates a functional, interpretable, and extensible prototype.
4. Ensure the synthesized prompt emphasizes alignment with product goals, UX findings, statistical rigor, architectural scalability, and roadmap milestones.

Deliver the final output as a single, well‑formed prompt ready to be passed into AIStudio.
\n\n"""
                               f"Product Plan: {response}\n\n"
                               f"UX Research: {ux_response}\n\n"
                            f"Statistical Requirements: {stats_analysis}"
                            f"Architecture and effort estimate: {arch_design}"
                            f"User Stories and Acceptance Criteria: {user_stories}"
                            f"UX Researcher Approval: {ux_ok}"
                            f"Statistician Approval: {stats_ok}"
                            f"Architect Approval: {arch_ok}", 
                            num_reflections=num_reflections)

🤔 Reflecting 2 time(s)...
🤔 Reflecting 2 time(s)...
🤔 Reflecting 2 time(s)...


In [55]:
print(strategy_prompt)

Of course. This is the critical next step: translating our comprehensive strategic plan for "The Deal Room" into a well-defined, sprint-ready backlog for the engineering team.

Based on our detailed product plan, V1 PRD, and the specific, actionable feedback from UX research, statistical analysis, and architecture, I have synthesized and refined the user stories. This backlog directly incorporates the key recommendations from our cross-functional partners and the improvements identified in our reflection. It includes agile best practices like a pre-development "Spike" for high-risk items and dedicated stories for all user journeys, including empty states.

Before we dive in, a note on our process: this backlog represents the "what" and the "why." To ensure quality and alignment, every story must meet our **Definition of Ready** (e.g., designs attached in Figma, acceptance criteria finalized) before being brought into a sprint. Upon completion, it must meet our **Definition of Done** (e

In [56]:
print(engg_driven_prompt)

Of course. Based on our comprehensive product strategy, the detailed V1 execution plan, and the critical feedback from our cross-functional partners and internal reflections, I am providing a complete system architecture and development plan for "The Deal Room."

This plan is engineered to deliver the focused V1 described in the user stories while creating the essential technical foundation for the ambitious V2/V3 roadmap. It directly incorporates the non-negotiable requirements for multi-tenancy, a limited stakeholder view, functional templates, and a causal-ready data model.

---

### 1. System Architecture Overview

The V1 architecture will be a **pragmatic, well-structured monolith** deployed on AWS. This approach prioritizes V1 development speed and simplicity, which is ideal for validating our core product hypotheses. The architecture is designed with clear logical boundaries that will allow for future decomposition into microservices once the product is validated and scaling bec

In [57]:
print(pm_driven_prompt)

Of course. Synthesizing our entire planning process into a single, comprehensive prompt for a generative AI tool like Google AIStudio is the perfect way to ensure all strategic, user-centric, and technical requirements are translated into a coherent prototype.

Based on our collaborative work and the specific improvements identified during our reflection, I have generated an enhanced, production-grade prompt. This version provides greater technical specificity, incorporates accessibility as a core principle, and adds clear acceptance criteria for the prototype itself to ensure the AI's output is testable, extensible, and precisely aligned with our goals.

---

### **Prompt for Google AIStudio: High-Fidelity Prototype Generation for "The Deal Room"**

**Objective:** Generate a high-fidelity, functional prototype for a new B2B SaaS product called "The Deal Room." This prototype will serve as the primary tool for user testing, internal validation, and the foundational codebase for our V1 

In [58]:
ux_driven_prompt, ux_memory_id = ux_researcher(f"""Your task is to generate a single, comprehensive prompt that can be fed into Google AIStudio. 
This prompt should synthesize the following inputs into a coherent specification for prototype generation using GenAI/vibecoding:

- Overall Product Plan
- UX Research
- Statistical Requirements
- Architecture Design and Effort Estimate
- User Stories and Acceptance Criteria
- Approvals (UX Researcher, Statistician, Architect)

When constructing the AIStudio prompt:
1. Integrate all inputs into a structured, readable format that AIStudio can consume directly.
2. Explicitly require that the prototype be responsive across both web and mobile platforms.
3. Frame the output requirements so AIStudio generates a functional, interpretable, and extensible prototype.
4. Ensure the synthesized prompt emphasizes alignment with product goals, UX findings, statistical rigor, architectural scalability, and roadmap milestones.

Deliver the final output as a single, well‑formed prompt ready to be passed into AIStudio.
\n\n"""
                               f"Product Plan: {response}\n\n"
                               f"UX Research: {ux_response}\n\n"
                            f"Statistical Requirements: {stats_analysis}"
                            f"Architecture and effort estimate: {arch_design}"
                            f"User Stories and Acceptance Criteria: {user_stories}"
                            f"UX Researcher Approval: {ux_ok}"
                            f"Statistician Approval: {stats_ok}"
                            f"Architect Approval: {arch_ok}", 
                            num_reflections=num_reflections)

🤔 Reflecting 2 time(s)...


In [59]:
print(ux_driven_prompt)

Of course. This is the critical next step: translating our comprehensive strategic plan for "The Deal Room" into a well-defined, sprint-ready backlog.

As the UX Researcher, I have reviewed these user stories and acceptance criteria in detail, and I am absolutely okay with them. This is a significantly more robust and mature backlog. The inclusion of a pre-development "Spike" for technical investigation, a dedicated onboarding epic with a clear time-to-value goal, and thoughtful "unhappy path" acceptance criteria demonstrates a strong, user-centric approach to agile development.

My assessment is that these user stories are an excellent foundation for V1. My feedback will focus on layering our comprehensive research plan directly onto this backlog. This plan is designed to de-risk our core assumptions, ensure we build with confidence, and deliver a product that is not just functional, but genuinely usable, valuable, and trusted.

---

### **Overall Assessment**

**Verdict: Yes, this is